In [4]:
import os
import re
import pandas as pd
from collections import defaultdict

# List of folders to process
folders = [
    "realgemini20_batching_and_splitting",
    "realgemini25_batching_and_splitting",
    "realgpt41_batching_and_splitting",  # Add more folders as needed
]

base_path = os.getcwd() # Adjust this if the full path differs

# Structure: {folder: {model: {type: [costs]}}}
all_costs = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for folder in folders:
    file_path = os.path.join(base_path, folder, "cost_log.txt")
    if not os.path.exists(file_path):
        print(f"Skipping missing file: {file_path}")
        continue

    with open(file_path, "r") as f:
        for line in f:
            match = re.search(r"^(.*?)\s+\|\s+.*?\|\s+(Non-Batched|Strict-Batched|Spread-Batched)\s+\|.*?Cost:\s+\$(\d+\.\d+)", line)
            if match:
                model, batch_type, cost = match.groups()
                model = model.strip()
                cost = float(cost)
                all_costs[folder][model][batch_type].append(cost)

# Print results
for folder, folder_data in all_costs.items():
    print(f"\n Folder: {folder}")
    for model, batch_data in folder_data.items():
        print(f"  🔹 Model: {model}")
        for batch_type, costs in batch_data.items():
            avg_cost = sum(costs) / len(costs)
            print(f"    {batch_type}: Average Cost = ${avg_cost:.6f}")



 Folder: realgemini20_batching_and_splitting
  🔹 Model: google/gemini-2.0-flash-001
    Non-Batched: Average Cost = $0.018947
    Strict-Batched: Average Cost = $0.019202
    Spread-Batched: Average Cost = $0.023563

 Folder: realgemini25_batching_and_splitting
  🔹 Model: google/gemini-2.5-flash-preview
    Non-Batched: Average Cost = $0.028773
    Strict-Batched: Average Cost = $0.029247
    Spread-Batched: Average Cost = $0.035904

 Folder: realgpt41_batching_and_splitting
  🔹 Model: openai/gpt-4.1-mini
    Non-Batched: Average Cost = $0.064339
    Strict-Batched: Average Cost = $0.067153
    Spread-Batched: Average Cost = $0.082396


In [6]:
import os
import re
from collections import defaultdict, Counter

# List of folders to process
folders = [
    "realgemini20_batching_and_splitting",
    "realgemini25_batching_and_splitting",
    "realgpt41_batching_and_splitting",  # Add more folders as needed
]

base_path = os.getcwd()  # Adjust if your base path is different

# Structure: {folder: {model: {batch_type: [times]}}}
all_times = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for folder in folders:
    file_path = os.path.join(base_path, folder, "time_log.txt")
    if not os.path.exists(file_path):
        print(f"⏱️ Skipping missing file: {file_path}")
        continue

    with open(file_path, "r") as f:
        for line in f:
            match = re.search(r"^(.*?)\s+\|\s+.*?\|\s+(Non-Batched|Strict-Batched|Spread-Batched)\s+\|\s+Time Taken:\s+([\d.]+)", line)
            if match:
                model, batch_type, time_taken = match.groups()
                model = model.strip()
                time_taken = float(time_taken)
                all_times[folder][model][batch_type].append(time_taken)

# Print average time results
for folder, folder_data in all_times.items():
    print(f"\n📁 Folder: {folder}")
    for model, batch_data in folder_data.items():
        print(f"  🔹 Model: {model}")
        for batch_type, times in batch_data.items():
            avg_time = sum(times) / len(times)
            print(f"    {batch_type}: Average Time Taken = {avg_time:.2f} seconds")



📁 Folder: realgemini20_batching_and_splitting
  🔹 Model: google/gemini-2.0-flash-001
    Non-Batched: Average Time Taken = 20.00 seconds
    Strict-Batched: Average Time Taken = 11.14 seconds
    Spread-Batched: Average Time Taken = 12.05 seconds

📁 Folder: realgemini25_batching_and_splitting
  🔹 Model: google/gemini-2.5-flash-preview
    Non-Batched: Average Time Taken = 22.65 seconds
    Strict-Batched: Average Time Taken = 12.52 seconds
    Spread-Batched: Average Time Taken = 12.41 seconds

📁 Folder: realgpt41_batching_and_splitting
  🔹 Model: openai/gpt-4.1-mini
    Non-Batched: Average Time Taken = 51.06 seconds
    Strict-Batched: Average Time Taken = 29.50 seconds
    Spread-Batched: Average Time Taken = 29.25 seconds


In [8]:
#folder path where you have your misclassified files
folder_path = "misclassified"

pattern = r"TRUE: (.*?) --> PRED: (.*)"
data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        misclass_counter = Counter()
        
        with open(file_path, "r") as file:
            for line in file:
                match = re.search(pattern, line)
                if match:
                    true_label, pred_label = match.groups()
                    true_label = true_label.strip()
                    pred_label = pred_label.strip()
                    if true_label != pred_label:
                        key = f"{true_label} → {pred_label}"
                        misclass_counter[key] += 1

        # Add to data list
        if misclass_counter:
            for misclass, count in sorted(misclass_counter.items(), key=lambda x: -x[1]):
                true_label, pred_label = misclass.split(" → ")
                data.append({
                    "Filename": filename,
                    "True Label": true_label,
                    "Predicted Label": pred_label,
                    "Count": count
                })
        else:
            data.append({
                "Filename": filename,
                "True Label": None,
                "Predicted Label": None,
                "Count": 0
            })

# Export to Excel
df = pd.DataFrame(data)
excel_file = "misclassified/misclassification_summary.xlsx"
df.to_excel(excel_file, index=False)

print(f"Exported to: {excel_file}")


Exported to: misclassified/misclassification_summary.xlsx
